In [1]:
import pandas as pd
import imdb
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import re
import urllib.request
from twilio.rest import Client

In [2]:
# Create an instance of the IMDb class
ia = imdb.IMDb()

In [3]:
# Set up your Twilio account
account_sid = 'ACcf747aac8a00c40c4337cd9811840373'
auth_token = 'a18fe1fbf64e8ad2ed1072c2b42d8cd0'
client = Client(account_sid, auth_token)

In [4]:
# Set the URL of the movie website you want to scrape
url =  'https://nzdworld.com/category/movies/page/1/'

In [5]:
# Set the minimum IMDB rating for the movies you want to receive notifications for
min_rating = 7.5

# Set your WhatsApp number
whatsapp_number = 'whatsapp:+2348104266501'

In [ ]:
page = 1
movies_list = []


while True:

    
    url =  f'https://nzdworld.com/category/movies/page/{page}/'
    
    response = requests.get(url)

      # Check if the response status code is not 200
    if response.status_code != 200:
        print(f'Response status code: {response.status_code}. Breaking out of loop.')
        break
        
        
    
    # Parse the HTML using BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')
        
    # Find all <h3> tags with class "g1-gamma g1-gamma-1st entry-title"
    h3_tags = soup.find_all('h3', class_='g1-gamma g1-gamma-1st entry-title')

    # Create an empty list to store the movie names and release dates

    # Loop through the <h3> tags and extract the nested <a> tag
    for h3_tag in h3_tags:
        a_tag = h3_tag.find('a')
        href = a_tag['href']
        rel = a_tag['rel']
        text = a_tag.text
        if 'MOVIE' in text:  # Filter out TV series
            movie_text = text.split(':')[-1].strip()  # Extract movie name
            movie_name = movie_text.rsplit('(', 1)[0].strip()  # Remove release date from movie name
            release_date = movie_text.rsplit('(', 1)[1].replace(')', '')  # Extract release date

            response = requests.get(str(href))
            soup = BeautifulSoup(response.content, 'html.parser')
            a_tag = soup.find('a', text='DOWNLOAD MOVIE')
            link = a_tag.get('href')

            # Getting movie rating
            search_results = ia.search_movie(str(movie_name))
            # Get the first search result (assuming it's the correct movie)
            movie_id = search_results[0].getID()

            # Get the movie details using the ID
            movie = ia.get_movie(movie_id)

            # Get the movie rating
            rating = movie.get('rating')

            movies_list.append([movie_name, release_date, rating,  link])
            
    
    print(f'{page} completed')
    page += 1
    # Show progress bar
#     progress = tqdm(total=100)
#     progress.update(page*10)  # Update the progress bar by 10% for every page
#     progress.close()
    
    
    
# Create a pandas DataFrame from the movies_list
movies_df = pd.DataFrame(movies_list, columns=['Movie Name', 'Release Date', 'Rating', 'Link'])


1 completed
2 completed
3 completed
4 completed
5 completed
6 completed
7 completed
8 completed
9 completed
10 completed
11 completed
12 completed
13 completed
14 completed
15 completed
16 completed
17 completed
18 completed
19 completed
20 completed
21 completed


In [7]:
movies_df = pd.DataFrame(movies_list, columns=['Movie Name', 'Release Date', 'Rating', 'Link'])


In [ ]:
# movies_df.to_csv('Movies.csv')

In [8]:
movies_df

,Movie Name,Release Date,Rating,Link
0,Tetris,2023,7.2,https://downloadwella.com/2z7lkqd9vrew/Tetris....
1,Assassin,2023,3.7,https://downloadwella.com/szl2dhs5sdg3/Assassi...
2,Murder Mystery 2,2023,5.7,https://downloadwella.com/5aj6bp3r3rvj/Murder....
3,Creed III,2023,7.3,https://downloadwella.com/38joyoo1cjef/Creed.I...
4,A Picture of Her,2023,6.8,https://downloadwella.com/5h1pxlcccy1a/A.Pictu...
...,...,...,...,...
143,High Heat,2022,4.8,https://downloadwella.com/3h1dtrrw33qa/High.He...
144,Christmas in Miami,2021,7.4,https://loadedfiles.com/010abdc660fb8c62
145,Lifemark,2022,5.9,https://downloadwella.com/c2pxyj9wj7il/Lifemar...
146,Snow Day,2022,5.1,https://downloadwella.com/qnl6nxdwpcc1/Snow.Da...


In [ ]:
# # Convert DataFrame to string
# movies_text = movies_df[['Movie Name', 'Rating', 'Link']].to_string(index=False)

# # Send message via Twilio WhatsApp
# message = client.messages.create(
#     body=movies_text,
#     from_='whatsapp:+14155238886',  # Twilio WhatsApp number
#     to = 'whatsapp:+2349012389795'
# )

# print('Message sent:', message.sid)
